In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

2023-12-06 18:24:51.291786: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [3]:
DATASET_PORTION = 'full'

base_dir = f"{os.getcwd()}/dataset/{DATASET_PORTION}"
train_dir = f"{base_dir}/train_and_validation/train"
valid_dir = f"{base_dir}/train_and_validation/validation"
test_dir = f"{base_dir}/test"

`flow_from_directory()` expects to read from a series of directories at the specified path, where each directory is a class and each file within that directory is a data instance.

Below, for example, I'm reading in the train directory `./dataset/subset/train_and_validation/train` which contains the classes in sub-directories (ex `Apple___Apple_scab`), and within this sub-directory the image files are listed (ex. `Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.JPG`).

In [11]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training')


Found 56248 images belonging to 38 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    valid_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 3503 images belonging to 38 classes.


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(256,256,3)),
        layers.Conv2D(32, 3, padding='valid', activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(38, activation='softmax'),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 460800)            0

In [7]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(),#from_logits=True
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
)

In [9]:
model.fit(
    train_generator,
    steps_per_epoch=100,
    validation_data=validation_generator,
    batch_size=32,
    epochs=5,
    verbose=2)

Epoch 1/5
100/100 - 153s - loss: 1.7821 - accuracy: 0.4900 - val_loss: 1.7029 - val_accuracy: 0.5104 - 153s/epoch - 2s/step
Epoch 2/5
100/100 - 145s - loss: 1.4807 - accuracy: 0.5694 - val_loss: 1.5047 - val_accuracy: 0.5786 - 145s/epoch - 1s/step
Epoch 3/5
100/100 - 128s - loss: 1.2483 - accuracy: 0.6269 - val_loss: 1.3721 - val_accuracy: 0.5983 - 128s/epoch - 1s/step
Epoch 4/5
100/100 - 128s - loss: 1.1291 - accuracy: 0.6684 - val_loss: 1.2630 - val_accuracy: 0.6297 - 128s/epoch - 1s/step
Epoch 5/5
100/100 - 129s - loss: 1.0070 - accuracy: 0.6999 - val_loss: 1.2189 - val_accuracy: 0.6403 - 129s/epoch - 1s/step


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 33 images belonging to 38 classes.


In [19]:
model.evaluate(
    test_generator,
    batch_size=32,
    verbose=2)


2/2 - 0s - loss: 2.3268 - accuracy: 0.3939 - 489ms/epoch - 244ms/step


[2.3267996311187744, 0.39393940567970276]